<a href="https://colab.research.google.com/github/tarrantcarter/Data-Science-Python-Basics/blob/master/Profitable_App_Profiles_for_the_App_Store_and_Google_Play_Markets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Profitable App Profiles for the App Store and Google Play Markets

Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

# Opening and Exploring the Data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps is very time intensive and could require money to obtain. So first we'll try to seek out and analyze a free sample of data. Fortunately, these are two data sets that seem suitable for our purpose:

- [An Android dataset](https://www.kaggle.com/lava18/google-play-store-apps) that holds data on over ten thousand Android Apps from the Google Play Store. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
- [An Ios dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) that holds data on over seven thousand Ios Apps from the App Store. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

Let's start by opening the two data sets and then continue with exploring the data.

In [ ]:
from csv import reader

### The Google Play data set ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### The App Store data set ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In order to explore the data sets at a glance, we have written a function called explore_data() that we can use repeatedly and will allow us to easily look at a more readable slice of data from each data set. Also, we will add to our function the ability to determine the number or rows and columns from each data set.

In [ ]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))     

Let's take a quick look at the Android data set.

In [ ]:
print(android_header)
print('\n')
first_rows_android = explore_data(android,0,4, True)
print(first_rows_android)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13
None


We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

Now let's take a look at the App Store data set.

In [ ]:
print(ios_header)
print('\n')
first_rows_ios = explore_data(ios,0,4, True)
print(first_rows_ios)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16
None


We have 7197 iOS apps in this data set, and the columns that seem interesting are: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

# Deleting Faulty Data

Next we will begin looking for faulty data and deleting it from our database. After looking at the Goggle Play discussion on [kaggle](https://www.kaggle.com/lava18/google-play-store-apps/discussion/81460) it is suspected a row has faulty data. We will inspect to confirm.

In [ ]:
print(android[10472:10474])

[['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'], ['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']]


As expected the 'Life Made WI-Fi Touchscreen Photo Frame' app is probably missing its category in column[1]. 1.9 is not an app category and probably belongs to the rating category in column[2]. We will delete the row by referencing its index in the android database.

In [ ]:
print(len(android))
del android[10472]
print(len(android))

10841
10840


# Deleting Duplicate Data

## Part 1

Upon further reading of the [Goggle Play discussion board](https://www.kaggle.com/lava18/google-play-store-apps/discussion) it is clear that there are numerous duplicate entries for many of the apps. One of the apps discussed was 'Intstagram'. We will develope a simple test to see how many entries of 'Instagram' appear in the database. 

In [ ]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Next we will create two lists. Onre for duplicate apps and one for unique apps. We will loop through the android data base and append each list to either the duplicate or unique list.

In [ ]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of unique apps:', len(unique_apps))
print('\n')  
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:5])

Number of unique apps: 9659


Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


There are 1181 duplicate entries in the Android database. Upon further inspection of the 'Instagram' app, there are 4 entries that appear to be the same in all columns but one. That one column is number of ratings. We want to remove the duplicate entries by filtering out the least reviewed duplicates and keeping the most reviewed unique apps.  

To do that, we will:

- Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
- Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

## Part Two
Let's start by building the dictionary. 

In [ ]:
reviews_max = {}
for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews            

In a previous code cell, we found that there are 1,181 cases where an app occurs more than once, so the length of our dictionary (of unique apps) should be equal to the difference between the length of our data set and 1,181.

In [ ]:
print('Expected Length:', len(android) -1181)
print('Actual Length:', len(reviews_max))

Expected Length: 9659
Actual Length: 9659


Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

- We start by initializing two empty lists, android_clean and already_added.
- We loop through the android data set, and for every iteration:
- - We isolate the name of the app and the number of reviews.
- - We add the current row (app) to the android_clean list, and the app name (name) to the already_added list if:
- - - The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
- - - The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [ ]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])

    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)    

Now let's quickly explore the new data set, and confirm that the number of rows is 9,659.

In [ ]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


We have 9659 rows, just as expected.

# Deleting Non-English Data

## Part One

If you explore the data sets enough, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [ ]:
print(ios[813][1])
print(ios[6731][1])

print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these kind of apps, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in ord() function to find out the corresponding encoding number of each character.

In [ ]:
def is_english(string):
    for character in string:
        if ord(character) > 127:
            return False
    return True

In [ ]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


The function seems to work fine, but some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form.

If we're going to use the function we've created, we'll lose useful data since many English apps will be incorrectly labeled as non-English. 

## Part Two

To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [ ]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
     
    if non_ascii > 3:
        return False
    else:        
        return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))


True
False
True
True


The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the is_english() function to filter out the non-English apps for both data sets:

In [ ]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name) == True:
        android_english.append(app)
        
print(explore_data(android_english,0,5, True))  
print(len(android_clean) - len(android_english))

for app in ios:
    name = app[1]
    if is_english(name) == True:
        ios_english.append(app)
        
print(explore_data(ios_english,0,5, True))  
print(len(ios) - len(ios_english))


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9614
Number of columns: 13
None
45
['284882215', 'Facebook', '389879808', '

Only 45 Andriod apps were removed, but over 1000 Ios Apps were removed. We can see that we're left with 9614 Android apps and 6183 iOS apps. 

# Isolating Free Apps

Since our company only builds free apps, we need to remove the non-free apps from our databases. Below, we isolate the free apps for both our data sets.

In [ ]:
android_free = []

for app in android_english:
    if app[7] == '0':
        android_free.append(app)
        
ios_free = []

for app in ios_english:
    if app[4] == '0.0':
        ios_free.append(app)       
        
print(len(android_free))
print(len(ios_free))


    

8864
3222


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

# Most Common Apps by Genre

## Part One

Our goal is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

- Build a minimal Android version of the app, and add it to Google Play.
- If the app has a good response from users, we develop it further.
- If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. 


## Part Two
Let's begin the analysis by getting a sense of what are the most common genres for each market. For this we will need to identify the relevent columns. For the android database the columns "Category" and "Genres" look like a good starting point. The column "prime_genre" looks like our starting point for the ios database.  

Below we will make frequency tables of the previously mentioned columns to better understand which genres are most common.

In [ ]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        x = row[index]
        if x in table:
            table[x] += 1
        else:
            table[x] = 1
    
    table_percentage = {}
    for key in table:
        percentage = (table[key] / total) * 100 
        table_percentage[key] = percentage 
            
    return table_percentage

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

    

## Part Three

We start by examining the frequency table for the prime_genre column of the App Store data set.

In [ ]:
print(display_table(ios_free,11))  

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665
None


Over 58% of the app store apps are in the 'games' genre. This dwarfs 'entertainment' which accounts for 7.9% of apps. This is followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The big picture view is that the App Sore (at least the free and english apps) is dominated by apps designed for leasure. Leasure apps like (games, photo and video, social networking, entertainment) make up 4 of the top 5 cateogries for apps created, compared to the smaller percentage of pratical apps like (shopping, utlities, heath and fitness, productivity)

Looking at just this database we would recommend make an app in the games genre. It accounts for 58% of total apps. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer. We will have to test that idea later. 

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).

In [ ]:
display_table(android_free,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The genre family tops out at over 18.9% on the Google play store. Second is game at 9.7%, followed by Tools at 8.5%. Business account for only 4.6% of total apps, while lieftsyle apps come in around 3.9%.

The picture appears more nuanced on the Google Play store compared to the App Store. Leasure Apps don't appear to dominate like in the App Store dataset. However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids. So over 27% of apps on the Google Play store seemed game-oriented. 

We have one more Google Play column to analyze. 

In [ ]:
print(display_table(android_free,9)) 

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Tools top out the second Google Play genre column we analyzed at around 8.4%. This is followed by Entertainmentat 6.1%. 3 practical use apps round out the top 5 apps from this column. Those are education at 5.3&, Business at 4.6%, and Productivity at 3.9%. Suprisingly the most frequent category from the App Store game is not a catergory in this Google Play column. 

This comparison will have limitations because it is apparent that both platforms define categories differently. However, this second Google Play column 'Genre' seems to define categories more granular than the previous column 'Category'.

Our plan was to make an app that is profitble on the Google Play store for 6 months and then make a similiar Ios version. With that in mind games still seem like the safest app investment given this data. Family (which appears to be kids games) and Games are the two most popular genres in the Google Play 'Category' column, while they don't appear in the second. We're only looking for the bigger picture at the moment, so we'll only work with the 'Category' column moving forward.

We know that games account for over 58% of App Store apps, so if we make a successaful game (possibly kids game) on the Google Play store, it has a good chance of success on the App Store. 

Given the lack of agreement between databases it best to dive in a bit deeper to the see which apps are more popular apps and see it that will help narrow down our app idea. 



# Most Popular Apps by Genre on the App Store
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [ ]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        x = row[index]
        if x in table:
            table[x] += 1
        else:
            table[x] = 1
    
    table_percentage = {}
    for key in table:
        percentage = (table[key] / total) * 100 
        table_percentage[key] = percentage 
            
    return table_percentage


ios_genre = freq_table(ios_free,11)


for genre in ios_genre:
    total = 0
    len_genre = 0 
   
    for app in ios_free:
        genre_app = app[11]
        if genre_app == genre:
            user_rating = float(app[5])
            total += user_rating
            len_genre += 1

    avg_user_ratings = total / len_genre
    print(genre, ':', avg_user_ratings)




Sports : 23008.898550724636
Reference : 74942.11111111111
Games : 22788.6696905016
Book : 39758.5
Business : 7491.117647058823
Catalogs : 4004.0
Travel : 28243.8
News : 21248.023255813954
Weather : 52279.892857142855
Music : 57326.530303030304
Utilities : 18684.456790123455
Education : 7003.983050847458
Photo & Video : 28441.54375
Finance : 31467.944444444445
Lifestyle : 16485.764705882353
Shopping : 26919.690476190477
Productivity : 21028.410714285714
Navigation : 86090.33333333333
Social Networking : 71548.34905660378
Health & Fitness : 23298.015384615384
Entertainment : 14029.830708661417
Medical : 612.0
Food & Drink : 33333.92307692308


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [ ]:
for app in ios_free:
    if app[11] == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


A similiar pattern applies to Social Networking apps. Facebook has almost 3 million ratings, while Pinterest has over a million. 

In [ ]:
for app in ios_free:
    if app[11] == 'Social Networking':
        print(app[1], ':', app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

Reference shows some promise. It is dominated by the Bible with almost a million ratings, but it doesn't have as much competition as the social networking apps and isn't has top heavy as the navigation apps. 

Two reference app paths seem viable given this data:
- **We could make an app that takes an already existing popular book and add more features than just the book. This could include:**
- - an audio version or clips of the text
- - daily quotes from the book 
- - quizzes about the book
- - forums to discuss book details deeper
- **We could make an app that takes an already existing game and provide additional material to help the user's progress in the game. This could include:**
- - general tips and specific tips about beating a certain level or boss
- - detailed maps of the game's landscape
- - skins for that game
- - dictionary or glossary of terms specific for that game
- - daily quotes or challenges for that game

In [ ]:
for app in ios_free:
    if app[11] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market.

# Most Popular Apps by Genre on Google Play
For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [ ]:
display_table(android_free, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [ ]:
android_category = freq_table(android_free,1)

for category in android_category:
    total = 0
    len_category = 0 
   
    for app in android_free:
        category_app = app[1]
        if category_app == category:
            installs = app[5]
            installs = installs.replace('+', ' ')
            installs = installs.replace(',', '')
            total += float(installs)
            len_category += 1

    avg_installs = total / len_category
    print(category, ':', avg_installs)

FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SPORTS : 3638640.1428571427
TOOLS : 10801391.298666667
COMMUNICATION : 38456119.167247385
BEAUTY : 513151.88679245283
EVENTS : 253542.22222222222
BUSINESS : 1712290.1474201474
EDUCATION : 1833495.145631068
LIFESTYLE : 1437816.2687861272
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
BOOKS_AND_REFERENCE : 8767811.894736841
WEATHER : 5074486.197183099
COMICS : 817657.2727272727
DATING : 854028.8303030303
PARENTING : 542603.6206896552
FINANCE : 1387692.475609756
HEALTH_AND_FITNESS : 4188821.9853479853
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
GAME : 15588015.603248259
HOUSE_AND_HOME : 1331540.5616438356
ENTERTAINMENT : 11640705.88235294
VIDEO_PLAYERS : 24727872.452830188
TRAVEL_AND_LOCAL : 13984077.710144928
PRODUCTIVITY : 16787331.344927534
AUTO_AND_VEHICLES : 647317.8170731707
SHOPPING : 7036877.311557789
LIBRARIES_AND_DEMO : 638503.734939759
PERSONALIZATION : 5201482.6122448975
NEWS_AND_

We need to sort the genres by the highest average number of installs to better visualize the data. To do this we will make a dictionary with the Categories as the keys and the number of average installs as the values. Then we will use the sort function with a for loop to sort by highest to lowest number of installs by genre. 

In [ ]:
android_installs = {'ENTERTAINMENT' : 11640705.88235294,
'BEAUTY' : 513151.88679245283,
'SPORTS' : 3638640.1428571427,
'WEATHER' : 5074486.197183099,
'AUTO_AND_VEHICLES' : 647317.8170731707,
'HEALTH_AND_FITNESS' : 4188821.9853479853,
'HOUSE_AND_HOME' : 1331540.5616438356,
'LIFESTYLE' : 1437816.2687861272,
'GAME' : 15588015.603248259,
'MAPS_AND_NAVIGATION' : 4056941.7741935486,
'SOCIAL' : 23253652.127118643,
'NEWS_AND_MAGAZINES' : 9549178.467741935,
'SHOPPING' : 7036877.311557789,
'EVENTS' : 253542.22222222222,
'PHOTOGRAPHY' : 17840110.40229885,
'FOOD_AND_DRINK' : 1924897.7363636363,
'LIBRARIES_AND_DEMO' : 638503.734939759,
'ART_AND_DESIGN' : 1986335.0877192982,
'EDUCATION' : 1833495.145631068,
'BOOKS_AND_REFERENCE' : 8767811.894736841,
'COMICS' : 817657.2727272727,
'COMMUNICATION' : 38456119.167247385,
'TOOLS' : 10801391.298666667,
'MEDICAL' : 120550.61980830671,
'FINANCE' : 1387692.475609756,
'DATING' : 854028.8303030303,
'PARENTING' : 542603.6206896552,
'TRAVEL_AND_LOCAL' : 13984077.710144928,
'BUSINESS' : 1712290.1474201474,
'FAMILY' : 3695641.8198090694,
'PERSONALIZATION' : 5201482.6122448975,
'VIDEO_PLAYERS' : 24727872.452830188,
'PRODUCTIVITY' : 16787331.344927534}

sorted_android_installs = sorted(android_installs, key=android_installs.get, reverse=True)
for r in sorted_android_installs:
    print(r, android_installs[r])
    


COMMUNICATION 38456119.167247385
VIDEO_PLAYERS 24727872.452830188
SOCIAL 23253652.127118643
PHOTOGRAPHY 17840110.40229885
PRODUCTIVITY 16787331.344927534
GAME 15588015.603248259
TRAVEL_AND_LOCAL 13984077.710144928
ENTERTAINMENT 11640705.88235294
TOOLS 10801391.298666667
NEWS_AND_MAGAZINES 9549178.467741935
BOOKS_AND_REFERENCE 8767811.894736841
SHOPPING 7036877.311557789
PERSONALIZATION 5201482.6122448975
WEATHER 5074486.197183099
HEALTH_AND_FITNESS 4188821.9853479853
MAPS_AND_NAVIGATION 4056941.7741935486
FAMILY 3695641.8198090694
SPORTS 3638640.1428571427
ART_AND_DESIGN 1986335.0877192982
FOOD_AND_DRINK 1924897.7363636363
EDUCATION 1833495.145631068
BUSINESS 1712290.1474201474
LIFESTYLE 1437816.2687861272
FINANCE 1387692.475609756
HOUSE_AND_HOME 1331540.5616438356
DATING 854028.8303030303
COMICS 817657.2727272727
AUTO_AND_VEHICLES 647317.8170731707
LIBRARIES_AND_DEMO 638503.734939759
PARENTING 542603.6206896552
BEAUTY 513151.88679245283
EVENTS 253542.22222222222
MEDICAL 120550.6198083

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [ ]:
for app in android_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [ ]:
under_100_m = []

for app in android_free:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [ ]:
for app in android_free:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [ ]:
for app in android_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [ ]:
for app in android_free:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

The Books and Reference category looks promising on the Google Play store as well as the App store. We saw a similiar trend of popular apps focusing on one book like the Bible that we saw in the App store. However, we did not find support for our additional material game app. This is probably because the Google Play store defines those apps as games unlike the App store. 

# Conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We conclude that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries and dictionaries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.